In [16]:
#!pip install autogluon 
#!pip install ccxt 
#!pip install pandas-ta

In [19]:
from autogluon.timeseries import TimeSeriesDataFrame, TimeSeriesPredictor
import numpy as np 
import pandas as pd 
import pandas_ta as ta
import ccxt
from datetime import datetime

In [2]:
chart_df = pd.read_feather('BTC_USDT-4h.feather')
chart_df['date'] = pd.to_datetime(chart_df['date'])
chart_df['date'] = chart_df['date'].dt.tz_localize(None) 

# define targets 
close = chart_df.close.values 
targets = [] 

for i in range(len(close) - 1): 
    ret = (close[i+1] - close[i]) / close[i] * 100.0 
    targets.append(ret) 
targets.append(None) 
chart_df["targets"] = targets 

In [3]:
chart_df["item_id"] = "BTC-USDT"

In [4]:
chart_df

,date,open,high,low,close,volume,targets,item_id
0,2017-08-17 04:00:00,4261.48,4349.99,4261.32,4349.99,82.088865,1.777245,BTC-USDT
1,2017-08-17 08:00:00,4333.32,4485.39,4333.32,4427.30,63.619882,-1.693131,BTC-USDT
2,2017-08-17 12:00:00,4436.06,4485.39,4333.42,4352.34,174.562001,-0.622883,BTC-USDT
3,2017-08-17 16:00:00,4352.33,4354.84,4200.74,4325.23,225.109716,-0.928274,BTC-USDT
4,2017-08-17 20:00:00,4307.56,4369.69,4258.56,4285.08,249.769913,0.170592,BTC-USDT
...,...,...,...,...,...,...,...,...
13714,2023-11-22 12:00:00,36463.94,36650.00,36239.20,36481.14,6621.289880,2.217146,BTC-USDT
13715,2023-11-22 16:00:00,36481.15,37431.21,36372.58,37289.98,10097.378810,0.317404,BTC-USDT
13716,2023-11-22 20:00:00,37289.98,37861.10,37214.81,37408.34,10963.014440,-0.290443,BTC-USDT
13717,2023-11-23 00:00:00,37408.35,37535.10,37240.00,37299.69,4156.038070,0.186382,BTC-USDT


In [5]:
chart_df.dtypes

date       datetime64[ns]
open              float64
high              float64
low               float64
close             float64
volume            float64
targets           float64
item_id            object
dtype: object

In [6]:
# feature engineering 
chart_df.set_index(pd.DatetimeIndex(chart_df["date"]), inplace=True)
chart_df["bop"] = chart_df.ta.bop(lookahead=False)
chart_df["ebsw"] = chart_df.ta.ebsw(lookahead=False) 
chart_df["cmf"] = chart_df.ta.cmf(lookahead=False) 
chart_df["vwap"] = chart_df.ta.vwap(lookahead=False) 
chart_df["rsi/100"] = chart_df.ta.rsi(lookahead=False) / 100.0 
chart_df["high/low"] = chart_df["high"] / chart_df["low"] 
chart_df["low/open"] = chart_df["low"] / chart_df["open"] 
chart_df["hwma"] = chart_df.ta.hwma(lookahead=False) 
chart_df["hwma/close"] = chart_df["hwma"] / chart_df["close"] 
for l in range(1, 12): 
    for col in ["open", "high", "low", "close", "volume", "vwap"]: 
        val = chart_df[col].values 
        val_ret = [None for _ in range(l)] 
        for i in range(l, len(val)):
            if val[i-l] == 0: 
                ret = 1 
            else:
                ret = val[i] / val[i-l] 
            val_ret.append(ret) 
        chart_df["{}_change_{}".format(col, l)] = val_ret 
        
chart_df.drop(columns={"open", "high", "low", "close", "volume", "vwap", "hwma"}, inplace=True) 
chart_df.dropna(inplace=True) 

/tmp/ipykernel_2398/167117496.py:4: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  chart_df["ebsw"] = chart_df.ta.ebsw(lookahead=False)


In [7]:
train_data = TimeSeriesDataFrame.from_data_frame(
    chart_df, 
    id_column="item_id", 
    timestamp_column="date", 
)
train_data.head()

targets       bop      ebsw       cmf  \
item_id  timestamp                                                     
BTC-USDT 2017-08-23 16:00:00 -0.074325 -0.703979  0.000000  0.121501   
         2017-08-23 20:00:00 -0.010452 -0.206431  0.577350  0.097590   
         2017-08-24 00:00:00  0.966068 -0.279572  0.797082  0.046232   
         2017-08-24 04:00:00  1.172074  0.455473  0.983912  0.111185   
         2017-08-24 08:00:00  0.475964  0.477511  0.988062  0.104474   

                               rsi/100  high/low  low/open  hwma/close  \
item_id  timestamp                                                       
BTC-USDT 2017-08-23 16:00:00  0.524823  1.038033  0.970799    0.991733   
         2017-08-23 20:00:00  0.522657  1.026746  0.983880    0.997912   
         2017-08-24 00:00:00  0.522331  1.029263  0.985052    1.002920   
         2017-08-24 04:00:00  0.550261  1.021330  0.994363    0.999789   
         2017-08-24 08:00:00  0.582468  1.024323  0.999085    0.996411   

                              open_change_1  high_change_1  ...  \
item_id  timestamp                                          ...   
BTC-USDT 2017-08-23 16:00:00       1.004983       0.998542  ...   
         2017-08-23 20:00:00       0.978599       0.981000  ...   
         2017-08-24 00:00:00       1.002543       1.006198  ...   
         2017-08-24 04:00:00       0.991941       0.993600  ...   
         2017-08-24 08:00:00       1.009777       1.017546  ...   

                              low_change_10  close_change_10  \
item_id  timestamp                                             
BTC-USDT 2017-08-23 16:00:00       1.112373         1.065938   
         2017-08-23 20:00:00       1.197000         1.083182   
         2017-08-24 00:00:00       1.089313         1.063908   
         2017-08-24 04:00:00       1.080168         1.076483   
         2017-08-24 08:00:00       1.076951         1.054655   

                              volume_change_10  vwap_change_10  \
item_id  timestamp                                               
BTC-USDT 2017-08-23 16:00:00          1.522754        1.075381   
         2017-08-23 20:00:00          0.912660        1.102681   
         2017-08-24 00:00:00          0.557071        1.089171   
         2017-08-24 04:00:00          0.686778        1.083785   
         2017-08-24 08:00:00          1.074910        1.081851   

                              open_change_11  high_change_11  low_change_11  \
item_id  timestamp                                                            
BTC-USDT 2017-08-23 16:00:00        1.049637        1.046454       1.038863   
         2017-08-23 20:00:00        1.030000        1.040500       1.103235   
         2017-08-24 00:00:00        1.073518        1.086849       1.201474   
         2017-08-24 04:00:00        1.083068        1.065640       1.090747   
         2017-08-24 08:00:00        1.074310        1.065366       1.095909   

                              close_change_11  volume_change_11  \
item_id  timestamp                                                
BTC-USDT 2017-08-23 16:00:00         1.025167          1.725297   
         2017-08-23 20:00:00         1.065146          1.330740   
         2017-08-24 00:00:00         1.083068          0.738662   
         2017-08-24 04:00:00         1.074186          0.555535   
         2017-08-24 08:00:00         1.089100          0.956111   

                              vwap_change_11  
item_id  timestamp                            
BTC-USDT 2017-08-23 16:00:00        1.025898  
         2017-08-23 20:00:00        1.074368  
         2017-08-24 00:00:00        1.100511  
         2017-08-24 04:00:00        1.089970  
         2017-08-24 08:00:00        1.090613  

[5 rows x 74 columns]

In [8]:
train_data = train_data.convert_frequency(freq="4h")

In [9]:
train_data.freq

'4H'

In [10]:
predictor = TimeSeriesPredictor(
    prediction_length=1, 
    path="autogluon_BTC", 
    target="targets", 
    eval_metric="MAE" 
)

In [11]:
predictor.fit(
    train_data, 
    presets="best_quality", 
    time_limit=2400,
)

Beginning AutoGluon training... Time limit = 2400s
AutoGluon will save models to 'autogluon_BTC'
=================== System Info ===================
AutoGluon Version:  1.0.0
Python Version:     3.9.16
Operating System:   Linux
Platform Machine:   x86_64
Platform Version:   #46~22.04.1-Ubuntu SMP PREEMPT_DYNAMIC Wed Jun 7 15:06:04 UTC 20
CPU Count:          8
GPU Count:          1
Memory Avail:       40.69 GB / 44.08 GB (92.3%)
Disk Space Avail:   96394.02 GB / 459479.69 GB (21.0%)
Setting presets to: best_quality

Fitting with arguments:
{'enable_ensemble': True,
 'eval_metric': MAE,
 'hyperparameters': 'default',
 'known_covariates_names': [],
 'num_val_windows': 2,
 'prediction_length': 1,
 'quantile_levels': [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9],
 'random_seed': 123,
 'refit_every_n_windows': 1,
 'refit_full': False,
 'target': 'targets',
 'time_limit': 2400,
 'verbosity': 2}

Inferred time series frequency: '4H'
train_data contains missing values represented by NaN. They h

In [12]:
# inference
# load saved checkpoint 
predictor = TimeSeriesPredictor.load("autogluon_BTC")
predictor 

Loading predictor from path autogluon_BTC


In [37]:
# get inference data 
bitget = ccxt.bitget()
ohlcv = bitget.fetch_ohlcv("BTC/USDT:USDT", "4h")
chart_df = pd.DataFrame(ohlcv, columns=['timestamp', 'open', 'high', 'low', 'close', 'volume'])

In [38]:
chart_df.head()

,timestamp,open,high,low,close,volume
0,1701748800000,41822.3,41908.8,41387.0,41524.7,11206.916
1,1701763200000,41524.7,41762.7,41420.0,41725.0,8300.591
2,1701777600000,41725.0,42487.9,41693.0,42252.3,23501.413
3,1701792000000,42252.3,44179.0,42115.8,43821.9,46774.065
4,1701806400000,43821.9,44518.2,43427.5,44071.4,18375.522


In [39]:
def preprocess(df):
    bitget = ccxt.bitget()
    dates = df["timestamp"].values
    utc_timestamps = []
    for timestamp in dates:
        # Convert to ISO 8601 format
        date_string = bitget.iso8601(int(timestamp))
        # Convert to datetime object without timezone conversion
        date_object = datetime.strptime(date_string, "%Y-%m-%dT%H:%M:%S.%fZ")
        # Format to string if needed (optional)
        formatted_utc_time = date_object.strftime("%Y-%m-%d %H:%M:%S")
        utc_timestamps.append(formatted_utc_time)
    # Update DataFrame
    df["date"] = utc_timestamps
    df = df.drop(columns={"timestamp"})
    return df


In [40]:
chart_df = preprocess(chart_df)

In [41]:
chart_df

,open,high,low,close,volume,date
0,41822.3,41908.8,41387.0,41524.7,11206.916,2023-12-05 04:00:00
1,41524.7,41762.7,41420.0,41725.0,8300.591,2023-12-05 08:00:00
2,41725.0,42487.9,41693.0,42252.3,23501.413,2023-12-05 12:00:00
3,42252.3,44179.0,42115.8,43821.9,46774.065,2023-12-05 16:00:00
4,43821.9,44518.2,43427.5,44071.4,18375.522,2023-12-05 20:00:00
...,...,...,...,...,...,...
95,43688.1,43759.1,43309.9,43671.0,30322.093,2023-12-21 00:00:00
96,43671.0,43898.4,43469.0,43775.9,24829.901,2023-12-21 04:00:00
97,43775.9,44146.5,43600.0,43956.8,24980.397,2023-12-21 08:00:00
98,43956.8,44269.6,43375.9,43833.0,30167.370,2023-12-21 12:00:00


In [42]:
close = chart_df.close.values 
targets = [] 

for i in range(len(close) - 1): 
    ret = (close[i+1] - close[i]) / close[i] * 100.0 
    targets.append(ret) 
targets.append(None) 
chart_df["targets"] = targets

chart_df["item_id"] = "BTC-USDT"

In [43]:
chart_df.dropna(inplace=True)

In [44]:
chart_df.set_index(pd.DatetimeIndex(chart_df["date"]), inplace=True)
chart_df["bop"] = chart_df.ta.bop(lookahead=False)
chart_df["ebsw"] = chart_df.ta.ebsw(lookahead=False) 
chart_df["cmf"] = chart_df.ta.cmf(lookahead=False) 
chart_df["vwap"] = chart_df.ta.vwap(lookahead=False) 
chart_df["rsi/100"] = chart_df.ta.rsi(lookahead=False) / 100.0 
chart_df["high/low"] = chart_df["high"] / chart_df["low"] 
chart_df["low/open"] = chart_df["low"] / chart_df["open"] 
chart_df["hwma"] = chart_df.ta.hwma(lookahead=False) 
chart_df["hwma/close"] = chart_df["hwma"] / chart_df["close"] 
for l in range(1, 12): 
    for col in ["open", "high", "low", "close", "volume", "vwap"]: 
        val = chart_df[col].values 
        val_ret = [None for _ in range(l)] 
        for i in range(l, len(val)):
            if val[i-l] == 0: 
                ret = 1 
            else:
                ret = val[i] / val[i-l] 
            val_ret.append(ret) 
        chart_df["{}_change_{}".format(col, l)] = val_ret 
        
chart_df.drop(columns={"open", "high", "low", "close", "volume", "vwap", "hwma"}, inplace=True) 
chart_df.dropna(inplace=True) 

/tmp/ipykernel_2398/2034567706.py:3: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  chart_df["ebsw"] = chart_df.ta.ebsw(lookahead=False)


In [45]:
test_data = TimeSeriesDataFrame.from_data_frame(
    chart_df, 
    id_column="item_id", 
    timestamp_column="date", 
)
test_data.head()

targets       bop      ebsw       cmf  \
item_id  timestamp                                                     
BTC-USDT 2023-12-11 16:00:00  1.151530 -0.629207  0.000000 -0.045100   
         2023-12-11 20:00:00  1.194894  0.586099  0.577350 -0.010672   
         2023-12-12 00:00:00  0.402377  0.646982  0.796465 -0.010043   
         2023-12-12 04:00:00 -0.782206  0.247788  0.983458  0.057182   
         2023-12-12 08:00:00 -0.563570 -0.592198  0.986993 -0.011017   

                               rsi/100  high/low  low/open  hwma/close  \
item_id  timestamp                                                       
BTC-USDT 2023-12-11 16:00:00  0.238461  1.040697  0.962025    1.023053   
         2023-12-11 20:00:00  0.312758  1.019758  0.994425    1.002202   
         2023-12-12 00:00:00  0.381019  1.018498  0.998403    0.985162   
         2023-12-12 04:00:00  0.402788  1.016380  0.991380    0.977790   
         2023-12-12 08:00:00  0.375063  1.013324  0.991360    0.981029   

                              open_change_1  high_change_1  ...  \
item_id  timestamp                                          ...   
BTC-USDT 2023-12-11 16:00:00       0.987660       0.987809  ...   
         2023-12-11 20:00:00       0.975366       0.987929  ...   
         2023-12-12 00:00:00       1.011515       1.014307  ...   
         2023-12-12 04:00:00       1.011949       1.002741  ...   
         2023-12-12 08:00:00       1.004024       1.000984  ...   

                              low_change_10  close_change_10  \
item_id  timestamp                                             
BTC-USDT 2023-12-11 16:00:00       0.920941         0.930048   
         2023-12-11 20:00:00       0.925314         0.938103   
         2023-12-12 00:00:00       0.944402         0.954264   
         2023-12-12 04:00:00       0.946146         0.954287   
         2023-12-12 08:00:00       0.949962         0.949374   

                              volume_change_10  vwap_change_10  \
item_id  timestamp                                               
BTC-USDT 2023-12-11 16:00:00         11.661805        0.953862   
         2023-12-11 20:00:00          2.896797        0.950863   
         2023-12-12 00:00:00          1.395005        0.949583   
         2023-12-12 04:00:00          2.383983        0.951513   
         2023-12-12 08:00:00          5.351217        0.951783   

                              open_change_11  high_change_11  low_change_11  \
item_id  timestamp                                                            
BTC-USDT 2023-12-11 16:00:00        0.951574        0.949162       0.923414   
         2023-12-11 20:00:00        0.932881        0.942753       0.928506   
         2023-12-12 00:00:00        0.940758        0.952996       0.939714   
         2023-12-12 04:00:00        0.949312        0.955040       0.948964   
         2023-12-12 08:00:00        0.958103        0.957158       0.949934   

                              close_change_11  volume_change_11  \
item_id  timestamp                                                
BTC-USDT 2023-12-11 16:00:00         0.932881          5.345598   
         2023-12-11 20:00:00         0.940758          3.580116   
         2023-12-12 00:00:00         0.949312          2.767185   
         2023-12-12 04:00:00         0.958103          2.266447   
         2023-12-12 08:00:00         0.946822          2.783377   

                              vwap_change_11  
item_id  timestamp                            
BTC-USDT 2023-12-11 16:00:00        0.949582  
         2023-12-11 20:00:00        0.952559  
         2023-12-12 00:00:00        0.948716  
         2023-12-12 04:00:00        0.951852  
         2023-12-12 08:00:00        0.951713  

[5 rows x 74 columns]

In [48]:
autogluon_yhat = predictor.predict(test_data)

autogluon_yhat = autogluon_yhat["mean"].values[0]

Model not specified in predict, will default to the model with the best validation score: WeightedEnsemble


In [49]:
autogluon_yhat

0.2571840668624302